# Data Cleaning

The FARS dataset documents every fatal car accident that occurs in the United States. It contains many features describing each crash that can be useful for a variety of different studies. With my analysis focused on the amount of fatal car accidents, I do not have need for all of the variables. Instead, I query the following variables:
 - State
 - Case Number
 - Atmospheric Conditions
 - County
 - Accident Date
 - Day of Week
My initial analysis works with only the data from the years 2010-2015.   
   
As you can see later on in this notebook, the FARS data is messy and needs a lot of cleaning before analysis can be conducted. One specific area that needs to be adjusted is the unit of observation. FARS documents information about every fatal accident, but I interested in the amount of fatal crashes in a given county over a certain period of time. Specifically, I hypothesize that Daylight Savings Time (DST) has at least a week long effect. Thus, the major goal of this data cleaning is to produce a dataframe with the correct unit of observation and appropriate variables.   
   
#### Notebook Preparation   
   
I begin by reading in some libraries that will help with the data cleaning. Also, I create a standard state_conversion dictionary that will be used later to take the state number FARS uses and convert it to its corresponding state string.

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
state_conversion = {1:'Alabama',
2:'Alaska',
4:'Arizona',
5:'Arkansas',
6:'California',
8:'Colorado',
9:'Connecticut',
10:'Delaware',
11:'District of Columbia',
12:'Florida',
13:'Georgia',
15:'Hawaii',
16:'Idaho',
17:'Illinois',
18:'Indiana',
19:'Iowa',
20:'Kansas',
21:'Kentucky',
22:'Louisiana',
23:'Maine',
24:'Maryland',
25:'Massachusetts',
26:'Michigan',
27:'Minnesota',
28:'Mississippi',
29:'Missouri',
30:'Montana',
31:'Nebraska',
32:'Nevada',
33:'New Hampshire',
34:'New Jersey',
35:'New Mexico',
36:'New York',
37:'North Carolina',
38:'North Dakota',
39:'Ohio',
40:'Oklahoma',
41:'Oregon',
42:'Pennsylvania',
44:'Rhode Island',
45:'South Carolina',
46:'South Dakota',
47:'Tennessee',
48:'Texas',
49:'Utah',
50:'Vermont',
51:'Virginia',
53:'Washington',
54:'West Virginia',
55:'Wisconsin',
56:'Wyoming'}

## Convert FARS Data to DataFrame 

   
All data from FARS was queried and saved locally on my machine as CSV files to facilitate the transition to the notebook.

In [3]:
data16 = pd.read_csv("/Users/tristanmoser/files/tristan/488/project/NHSA/weather/w2016.csv")
data15 = pd.read_csv("/Users/tristanmoser/files/tristan/488/project/NHSA/weather/w2015.csv")
data14 = pd.read_csv("/Users/tristanmoser/files/tristan/488/project/NHSA/weather/w2014.csv")
data13 = pd.read_csv("/Users/tristanmoser/files/tristan/488/project/NHSA/weather/w2013.csv")
data12 = pd.read_csv("/Users/tristanmoser/files/tristan/488/project/NHSA/weather/w2012.csv")
data11 = pd.read_csv("/Users/tristanmoser/files/tristan/488/project/NHSA/weather/w2011.csv")
data10 = pd.read_csv("/Users/tristanmoser/files/tristan/488/project/NHSA/weather/w2010.csv")

Just as with the state variable, I convert each weather value to its corresponding string interpretation to gain more intuition to what is occuring for each crash.

In [5]:
convert = {1:'Clear', 2:'Rain', 3:'Sleet or Hail', 4:'Snow', 5:'Fog or Smoke', 6:'Severe Crosswinds', 
          7:'Blowing Sand or Dirt', 8:'Other', 10:'Cloudy', 11:'Blowing Snow', 12: 'Freezing Rain'}

In [6]:
#Replace weather variables
data16 = data16.replace({"atmcond": convert})
data15 = data15.replace({"atmcond": convert})
data14 = data14.replace({"atmcond": convert})
data13 = data13.replace({"atmcond": convert})
data12 = data12.replace({"atmcond": convert})
data11 = data11.replace({"atmcond": convert})
data10 = data10.replace({"atmcond": convert})

### Date Variables

   Because I know what year each of the datasets come from, I elimate the year from the "accident date" variable so that I can isolate the actual month-day combination. I also create variables to indicate which day and week of the year the crash occurs.

In [7]:
#Clean/Create date variables
def date_change(dat,x):
    dat['accdate'] = (dat['accdate']-x)/10000
    dat['year'] = x
    dat['week'] = 0
    dat['day'] = 0
    return dat

In [8]:
#Apply date changes
data16 = date_change(data16,2016)
data15 = date_change(data15,2015)
data14 = date_change(data14,2014)
data13 = date_change(data13,2013)
data12 = date_change(data12,2012)
data11 = date_change(data11,2011)
data10 = date_change(data10,2010)

In [9]:
#drop any null rows and sort dataframes by accident date
data16 = data16.dropna().sort_values(by='accdate')
data15 = data15.dropna().sort_values(by='accdate')
data14 = data14.dropna().sort_values(by='accdate')
data13 = data13.dropna().sort_values(by='accdate')
data12 = data12.dropna().sort_values(by='accdate')
data11 = data11.dropna().sort_values(by='accdate')
data10 = data10.dropna().sort_values(by='accdate')

In [10]:
dataframes = [data16,data15,data14,data13,data12,data11,data10]

In [11]:
#Fill day variable with day of the year
for dataframe in dataframes:
    #Isolate each unique date (All days of the year)
    df = dataframe['accdate'].unique()
    df = pd.Series(df)
    dfs = pd.DataFrame(columns=[['date','day']])
    dfs['date'] = df
    dfs['day'] = df.index
    #Convert date to numeric day of year
    change = {dfs['date'][ii]: dfs['day'][ii] + 1 for ii in range(len(dfs))}
    for ii in range(1,len(dataframe)):
        dataframe['day'][ii] = change[dataframe['accdate'][ii]]

In [12]:
#Check to make sure it functions correctly
data16.head()

,statenum,casenum,atmcond,county,accdate,dayofweek,week,day
9409,42.0,26.0,Clear,3.0,101.0,6.0,0,1
6290,6.0,560.0,Clear,59.0,101.0,6.0,0,1
21530,32.0,13.0,Clear,31.0,101.0,6.0,0,1
6909,48.0,95.0,Rain,29.0,101.0,6.0,0,1
23467,17.0,7.0,Clear,7.0,101.0,6.0,0,1


### County
   
   Another feature that needs to be addressed is what county the crash occurs in. FARS does provide this with their "county" variable, but there is a lot of overlap as the same county number can appear in different states. In order to make sure that each county is accounted for correctly, I create a geography variable named "geo" that creates a unique identifier for each county number inside each state.

In [13]:
dataframes = [data16,data15,data14,data13,data12,data11,data10]

In [14]:
#Create 'geo' identifer
for dataframe in dataframes:
    dataframe['geo'] = 0
    for ii in range(1,len(dataframe)):
        dataframe['geo'][ii] = dataframe['statenum'][ii]*1000 + dataframe['county'][ii]

In [15]:
#Check my work
data16.head()

,statenum,casenum,atmcond,county,accdate,dayofweek,week,day,geo
9409,42.0,26.0,Clear,3.0,101.0,6.0,0,1,42003
6290,6.0,560.0,Clear,59.0,101.0,6.0,0,1,6059
21530,32.0,13.0,Clear,31.0,101.0,6.0,0,1,32031
6909,48.0,95.0,Rain,29.0,101.0,6.0,0,1,48029
23467,17.0,7.0,Clear,7.0,101.0,6.0,0,1,17007


### Week of Year

   I now look at generating what week of the year the crash occurs in. There is no simple way to perform this task. I settled on creating various functions that take a start date and calculate where each week starts and ends. The start date input is the day of the week that the year starts on i.e. 3 is for Tuesday, 5 is for Thursday and 7 is for Saturday. 

In [16]:
dataframes = [data16,data15,data14,data13,data12,data11,data10]

In [17]:
#Each function is passed a dataframe and a start date
def week1_gen(dat,x):
    #Loop through every crash observation
    for ii in range(1,len(dat)):
        #Condition on value of the day of the year
        if dat['day'][ii] < x:
            dat['week'][ii] = 1
        elif ((dat['day'][ii] >= x) & (dat['day'][ii] < (x+7))):
            dat['week'][ii] = 2
        elif ((dat['day'][ii] >= (x+7)) & (dat['day'][ii] < (x+14))):
            dat['week'][ii] = 3
        elif ((dat['day'][ii] >= (x+14) & (dat['day'][ii] < (x+21)))):
            dat['week'][ii] = 4
        else:
            pass

def week2_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+21)) & (dat['day'][ii] < (x+28))):
            dat['week'][ii] = 5
        elif ((dat['day'][ii] >= (x+28)) & (dat['day'][ii] < (x+35))):
            dat['week'][ii] = 6
        elif ((dat['day'][ii] >= (x+35)) & (dat['day'][ii] < (x+42))):
            dat['week'][ii] = 7
        elif ((dat['day'][ii] >= (x+42)) & (dat['day'][ii] < (x+49))):
            dat['week'][ii] = 8
        else:
            pass

def week3_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+49)) & (dat['day'][ii] < (x+56))):
            dat['week'][ii] = 9
        elif ((dat['day'][ii] >= (x+56)) & (dat['day'][ii] < (x+63))):
            dat['week'][ii] = 10
        elif ((dat['day'][ii] >= (x+63)) & (dat['day'][ii] < (x+70))):
            dat['week'][ii] = 11
        elif ((dat['day'][ii] >= (x+70)) & (dat['day'][ii] < (x+77))):
            dat['week'][ii] = 12
        else:
            pass

def week4_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(11*7))) & (dat['day'][ii] < (x+(12*7)))):
            dat['week'][ii] = 13
        elif ((dat['day'][ii] >= (x+(12*7))) & (dat['day'][ii] < (x+(13*7)))):
            dat['week'][ii] = 14
        elif ((dat['day'][ii] >= (x+(13*7))) & (dat['day'][ii] < (x+(14*7)))):
            dat['week'][ii] = 15
        elif ((dat['day'][ii] >= (x+(14*7))) & (dat['day'][ii] < (x+(15*7)))):
            dat['week'][ii] = 16
        else:
            pass

def week5_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(15*7))) & (dat['day'][ii] < (x+(16*7)))):
            dat['week'][ii] = 17
        elif ((dat['day'][ii] >= (x+(16*7))) & (dat['day'][ii] < (x+(17*7)))):
            dat['week'][ii] = 18
        elif ((dat['day'][ii] >= (x+(17*7))) & (dat['day'][ii] < (x+(18*7)))):
            dat['week'][ii] = 19
        elif ((dat['day'][ii] >= (x+(18*7))) & (dat['day'][ii] < (x+(19*7)))):
            dat['week'][ii] = 20
        else:
            pass

def week6_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(19*7))) & (dat['day'][ii] < (x+(20*7)))):
            dat['week'][ii] = 21
        elif ((dat['day'][ii] >= (x+(20*7))) & (dat['day'][ii] < (x+(21*7)))):
            dat['week'][ii] = 22
        elif ((dat['day'][ii] >= (x+(21*7))) & (dat['day'][ii] < (x+(22*7)))):
            dat['week'][ii] = 23      
        elif ((dat['day'][ii] >= (x+(22*7))) & (dat['day'][ii] < (x+(23*7)))):
            dat['week'][ii] = 24
        else:
            pass

def week7_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(23*7))) & (dat['day'][ii] < (x+(24*7)))):
            dat['week'][ii] = 25
        elif ((dat['day'][ii] >= (x+(24*7))) & (dat['day'][ii] < (x+(25*7)))):
            dat['week'][ii] = 26
        elif ((dat['day'][ii] >= (x+(25*7))) & (dat['day'][ii] < (x+(26*7)))):
            dat['week'][ii] = 27
        elif ((dat['day'][ii] >= (x+(26*7))) & (dat['day'][ii] < (x+(27*7)))):
            dat['week'][ii] = 28
        else:
            pass

def week8_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(27*7))) & (dat['day'][ii] < (x+(28*7)))):
            dat['week'][ii] = 29
        elif ((dat['day'][ii] >= (x+(28*7))) & (dat['day'][ii] < (x+(29*7)))):
            dat['week'][ii] = 30
        elif ((dat['day'][ii] >= (x+(29*7))) & (dat['day'][ii] < (x+(30*7)))):
            dat['week'][ii] = 31
        elif ((dat['day'][ii] >= (x+(30*7))) & (dat['day'][ii] < (x+(31*7)))):
            dat['week'][ii] = 32
        else:
            pass

def week9_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(31*7))) & (dat['day'][ii] < (x+(32*7)))):
            dat['week'][ii] = 33
        elif ((dat['day'][ii] >= (x+(32*7))) & (dat['day'][ii] < (x+(33*7)))):
            dat['week'][ii] = 34
        elif ((dat['day'][ii] >= (x+(33*7))) & (dat['day'][ii] < (x+(34*7)))):
            dat['week'][ii] = 35
        elif ((dat['day'][ii] >= (x+(34*7))) & (dat['day'][ii] < (x+(35*7)))):
            dat['week'][ii] = 36
        else:
            pass

def week10_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(35*7))) & (dat['day'][ii] < (x+(36*7)))):
            dat['week'][ii] = 37
        elif ((dat['day'][ii] >= (x+(36*7))) & (dat['day'][ii] < (x+(37*7)))):
            dat['week'][ii] = 38
        elif ((dat['day'][ii] >= (x+(37*7))) & (dat['day'][ii] < (x+(38*7)))):
            dat['week'][ii] = 39
        elif ((dat['day'][ii] >= (x+(38*7))) & (dat['day'][ii] < (x+(39*7)))):
            dat['week'][ii] = 40
        else:
            pass

def week11_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(39*7))) & (dat['day'][ii] < (x+(40*7)))):
            dat['week'][ii] = 41
        elif ((dat['day'][ii] >= (x+(40*7))) & (dat['day'][ii] < (x+(41*7)))):
            dat['week'][ii] = 42
        elif ((dat['day'][ii] >= (x+(41*7))) & (dat['day'][ii] < (x+(42*7)))):
            dat['week'][ii] = 43
        elif ((dat['day'][ii] >= (x+(42*7))) & (dat['day'][ii] < (x+(43*7)))):
            dat['week'][ii] = 44
        else:
            pass
def week12_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(43*7))) & (dat['day'][ii] < (x+(44*7)))):
            dat['week'][ii] = 45
        elif ((dat['day'][ii] >= (x+(44*7))) & (dat['day'][ii] < (x+(45*7)))):
            dat['week'][ii] = 46
        elif ((dat['day'][ii] >= (x+(45*7))) & (dat['day'][ii] < (x+(46*7)))):
            dat['week'][ii] = 47
        elif ((dat['day'][ii] >= (x+(46*7))) & (dat['day'][ii] < (x+(47*7)))):
            dat['week'][ii] = 48
        else:
            pass
def week13_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(47*7))) & (dat['day'][ii] < (x+(48*7)))):
            dat['week'][ii] = 49
        elif ((dat['day'][ii] >= (x+(48*7))) & (dat['day'][ii] < (x+(49*7)))):
            dat['week'][ii] = 50
        elif ((dat['day'][ii] >= (x+(49*7))) & (dat['day'][ii] < (x+(50*7)))):
            dat['week'][ii] = 51
        elif ((dat['day'][ii] >= (x+(50*7))) & (dat['day'][ii] < (x+(51*7)))):
            dat['week'][ii] = 52
        else:
            pass
def week14_gen(dat,x):
    for ii in range(1,len(dat)):
        if ((dat['day'][ii] >= (x+(51*7)))):
            dat['week'][ii] = 53

In [18]:
#List all functions to loop over for each dataframe
weeks = [week1_gen,week2_gen,week3_gen,week4_gen,week5_gen,week6_gen,week7_gen,week8_gen,
         week9_gen,week10_gen,week11_gen,week12_gen,week13_gen,week14_gen]

In [19]:
#Loop
for week in weeks:
        week(data16,3)
        week(data15,4)
        week(data14,5)
        week(data13,6)
        week(data12,8)
        week(data11,2)
        week(data10,3)

In [21]:
#Check work
data16.tail()

,statenum,casenum,atmcond,county,accdate,dayofweek,week,day,geo
2139,55.0,492.0,Cloudy,53.0,1231.0,7.0,53,366,55053
19745,37.0,1252.0,Clear,49.0,1231.0,7.0,53,366,37049
4040,42.0,1090.0,Clear,89.0,1231.0,7.0,53,366,42089
25823,48.0,3429.0,Rain,287.0,1231.0,7.0,53,366,48287
16693,48.0,3456.0,Cloudy,109.0,1231.0,7.0,53,366,48109


With week now created, I create an even more unique identifier for a county-week that will be used as the unit of observation in the finalized dataframe.

In [26]:
#Structure data [state][county][week]
data16['unit'] = data16['geo']*1000 + data16['week']
data15['unit'] = data15['geo']*1000 + data15['week']
data14['unit'] = data14['geo']*1000 + data14['week']
data13['unit'] = data13['geo']*1000 + data13['week']
data12['unit'] = data12['geo']*1000 + data12['week']
data11['unit'] = data11['geo']*1000 + data11['week']
data10['unit'] = data10['geo']*1000 + data10['week']

In [20]:
dataframes = [data16,data15,data14,data13,data12,data11,data10]

I combine all of the dataframes into one so that all unique counties can be looped over. This will ensure that all county-weeks have observations even if they do not have a fatal accident in every year. 

In [22]:
weekstep = pd.concat(dataframes)

The following function will create a finalized dataframe for each year of data. Due to the relatively large size of the dataframes and the fact that the function loops over thousands of county-week combinations, the function takes some time to complete.

In [29]:
def final(dat,year):
    #Create lists that will be converted to dataframe columns
    weather = []
    crash = []
    state = []
    years2 = []
    weeks1 = []
    units1 = []
    #Loop over every county for every week
    for county in weekstep['geo'].unique():
        for week in range(1,54):
            #Isolate only rows with crashes in given county
            data1 = dat.drop(dat[((dat['geo'] != county))].index)
            #Isolate only rows in corresponding year and week
            data2 = data1.drop(data1[(data1['year'] != year)].index)
            data3 = data2.drop(data2[(data2['week'] != week)].index)
            #How many observations are found is the number of fatal crashes
            counts = len(data3)
            #The most common weather for that week is stored
            values = data3['atmcond'].value_counts().keys().tolist()
            #Take the state that the county is in
            stat = data3['statenum'].value_counts().keys().tolist()
            #Store given inputs
            yea = year
            wee = week
            uni = county
            #Add county to list
            units1.append(uni)
            #Add most frequent weather to list
            try:
                weather.append(values[0])
            except IndexError:
                weather.append('None')
            #Add state to list
            try:
                state.append(stat[0])
            except IndexError:
                state.append('None')
            #Add week, year and crash counts to corresponding lists
            weeks1.append(wee)
            years2.append(yea)
            crash.append(counts)
    #Create dataframe will full lists
    final = pd.DataFrame(columns=[['unit','weather']])
    final['unit'] = units1
    final['weather'] = weather
    final['crashes'] = crash
    final['state'] = state
    final['year'] = years2
    final['week'] = weeks1
    return final

With the final dataframes built, there is still some cleaning to do. Specifically, I create functions that clean the state and weather columns. For the state column, I convert all of the state values to their string form.   
   
The weather column needs a little more thought. Because every county-week does not have a fatal crash recorded, I am not given the weather information for that county during that week. To avoid having a null value or assuming that the weather is clear for that week, I take the weather value for the week just before the null observation and use that as the best estimate.

In [36]:
def clean_state(dat):
    dat['conv'] = dat['unit']/1000
    dat['unit'] = dat['unit'].astype(str)
    for ii in range(len(dat)):
        #Distinguish between single and double digit state numbers
        if dat['conv'][ii] < 10:
            dat['state'][ii] = dat['unit'][ii][0:1]
        else:
            dat['state'][ii] = dat['unit'][ii][0:2]
    dat['state'] = dat['state'].astype(int)
    #Convert number state values to string names
    for ii in range(len(dat)):
        if dat['state'][ii] != 0:
            dat['state'][ii] = state_conversion[dat['state'][ii]]
        else:
            pass
    return dat

In [48]:
def clean_weather(dat):
    for ii in range(len(dat)):
        #Find weeks with no crash records
        if dat['weather'][ii] == 'None':
            try:
                dat['weather'][ii] = dat['weather'][ii-1]
            except KeyError:
                dat['weather'][ii] = 'Clear'
        else:
            pass
    return dat

In [41]:
final15 = final(data15,2015)
final15 = clean_state(final15)
final15 = clean_weather(final15)

In [42]:
#Check work
final15.head(15)

,unit,weather,crashes,state,year,week,conv
0,42003,Other,1,Pennsylvania,2015,1,42.003
1,42003,Other,0,Pennsylvania,2015,2,42.003
2,42003,Other,0,Pennsylvania,2015,3,42.003
3,42003,Clear,1,Pennsylvania,2015,4,42.003
4,42003,Clear,1,Pennsylvania,2015,5,42.003
5,42003,Clear,0,Pennsylvania,2015,6,42.003
6,42003,Clear,1,Pennsylvania,2015,7,42.003
7,42003,Snow,2,Pennsylvania,2015,8,42.003
8,42003,Snow,0,Pennsylvania,2015,9,42.003
9,42003,Snow,0,Pennsylvania,2015,10,42.003


In [44]:
final14 = final(data14,2014)
final14 = clean_state(final14)
final14 = clean_weather(final14)

In [49]:
final13 = final(data13,2013)
final13 = clean_state(final13)
final13 = clean_weather(final13)

In [57]:
final12 = final(data12,2012)
final12 = clean_state(final12)
final12 = clean_weather(final12)

In [60]:
final11 = final(data11,2011)
final11 = clean_state(final11)
final11 = clean_weather(final11)

In [70]:
final10 = final(data10,2010)
final10 = clean_state(final10)
final10 = clean_weather(final10)

There are some other datasets that I will combine with my finalized dataframe that include the following information:
 - Financial: State expenditures and taxes from Urban Institute
 - Population: State populations from the US census

In [79]:
tax = pd.read_csv('/Users/tristanmoser/files/tristan/488/project/NHSA/weather/state_exp.csv')
pop = pd.read_csv('/Users/tristanmoser/files/tristan/488/project/NHSA/weather/Populations.csv')

In [81]:
#Isolate taxes by year
tax15 = tax[tax['Year'] == '2015']
tax14 = tax[tax['Year'] == '2014']
tax13 = tax[tax['Year'] == '2013']
tax12 = tax[tax['Year'] == '2012']
tax11 = tax[tax['Year'] == '2011']
tax10 = tax[tax['Year'] == '2010']

In [82]:
#Merge dataframes by state as the tax data is at the state level
total15 = pd.merge(final15,tax15,on='state')
total14 = pd.merge(final14,tax14,on='state')
total13 = pd.merge(final13,tax13,on='state')
total12 = pd.merge(final12,tax12,on='state')
total11 = pd.merge(final11,tax11,on='state')
total10 = pd.merge(final10,tax10,on='state')

In [91]:
#Merge dataframes by state and year as the population data is at the state level
totals15 = pd.merge(total15,pop[pop['Year']==2015],on='state')
totals14 = pd.merge(total14,pop[pop['Year']==2014],on='state')
totals13 = pd.merge(total13,pop[pop['Year']==2013],on='state')
totals12 = pd.merge(total12,pop[pop['Year']==2012],on='state')
totals11 = pd.merge(total11,pop[pop['Year']==2011],on='state')
totals10 = pd.merge(total10,pop[pop['Year']==2010],on='state')

In [96]:
#Check work
totals15.tail(5)

,unit,weather,crashes,state,year,week,conv,Year_x,Tax_On_Fuel,Tax_on_License,Tot_Hwy_Exp,Year_y,Population
166309,50017,Clear,0,Vermont,2015,49,50.017,2015,"$85,619","$111,527","$501,722",2015,626088
166310,50017,Clear,0,Vermont,2015,50,50.017,2015,"$85,619","$111,527","$501,722",2015,626088
166311,50017,Clear,0,Vermont,2015,51,50.017,2015,"$85,619","$111,527","$501,722",2015,626088
166312,50017,Clear,0,Vermont,2015,52,50.017,2015,"$85,619","$111,527","$501,722",2015,626088
166313,50017,Clear,0,Vermont,2015,53,50.017,2015,"$85,619","$111,527","$501,722",2015,626088


Most of the data is ready for my analysis. Now, I create the variable that means the most to this study: DST indicator. I create this with a simple function based on the week of Spring DST for the given year.   
   
I also create the variable "prox" that denotes how many weeks from the DST transition week the observation is. Note that this can be negative.

In [109]:
def dst(dat,week):
    dat['DST'] = 0
    for ii in range(len(dat)):
        #Condition on week of DST transition
        if dat['week'][ii] == week:
            dat['DST'][ii] = 1
    dat['prox'] = 0
    for ii in range(len(dat)):
        dat['prox'][ii] = dat['week'][ii] - week
    return dat

In [110]:
#Create new variables
tota15 = dst(totals15,11.0)
tota14 = dst(totals14,11.0)
tota13 = dst(totals13,11.0)
tota12 = dst(totals12,11.0)
tota11 = dst(totals11,12.0)
tota10 = dst(totals10,12.0)

In [112]:
#Check work
tota10.head(15)

,unit,weather,crashes,state,year,week,conv,Year_x,Tax_On_Fuel,Tax_on_License,Tot_Hwy_Exp,Year_y,Population,DST,prox
0,42003,Clear,0,Pennsylvania,2010,1,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-11
1,42003,Rain,2,Pennsylvania,2010,2,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-10
2,42003,Clear,1,Pennsylvania,2010,3,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-9
3,42003,Clear,0,Pennsylvania,2010,4,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-8
4,42003,Clear,0,Pennsylvania,2010,5,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-7
5,42003,Clear,0,Pennsylvania,2010,6,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-6
6,42003,Clear,0,Pennsylvania,2010,7,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-5
7,42003,Snow,2,Pennsylvania,2010,8,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-4
8,42003,Clear,2,Pennsylvania,2010,9,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-3
9,42003,Clear,0,Pennsylvania,2010,10,42.003,2010,"$2,020,099","$2,546,905","$6,738,126",2010,12712343,0,-2


In [113]:
finals = [tota15,tota14,tota13,tota12,tota11,tota10]

In [114]:
totals = pd.concat(finals)

In [115]:
#Drop unnecessary duplicate variables
totals = totals.drop(['Year_x','Year_y'],axis=1)

There are a few states in my sample that do not follow DST and are therefore omitted from the study. The District of Columbia is also omitted.

In [116]:
#Drop non-DST states
totals = totals.drop(totals[(totals['state'] == 'Arizona')].index)
totals = totals.drop(totals[(totals['state'] == 'District of Columbia')].index)
totals = totals.drop(totals[(totals['state'] == 'Hawaii')].index)

#Drop anomaly of week 0
totals = totals.drop(totals[(totals['week'] == 0)].index)

In [118]:
#Check work
totals.tail()

,unit,weather,crashes,state,year,week,conv,Tax_On_Fuel,Tax_on_License,Tot_Hwy_Exp,Population,DST,prox
166309,50017,Clear,0,Vermont,2010,49,50.017,"$99,278","$106,558","$337,990",625982,0,37
166310,50017,Clear,0,Vermont,2010,50,50.017,"$99,278","$106,558","$337,990",625982,0,38
166311,50017,Clear,0,Vermont,2010,51,50.017,"$99,278","$106,558","$337,990",625982,0,39
166312,50017,Clear,0,Vermont,2010,52,50.017,"$99,278","$106,558","$337,990",625982,0,40
166313,50017,Clear,0,Vermont,2010,53,50.017,"$99,278","$106,558","$337,990",625982,0,41


In [120]:
#Save data
totals.to_csv("/Users/tristanmoser/files/tristan/488/project/NHSA/weather/Totals2wks.csv",index=False)

The data is now ready for analysis!